Objective: Modify Max NPS Uplift for Negative Sentiment

In [1]:
import os
import csv
import math
import pandas as pd
import numpy as np
import datetime
import xlwt
import glob
import pyexcel as p
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
import os

from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, OneClassSVM
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

import xgboost as xgb
import json

import matplotlib.pyplot as plt
%matplotlib inline


import scipy
from scipy import stats
import scipy.stats as stats

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.arima_model import ARIMA

from statsmodels.graphics.mosaicplot import mosaic
from statsmodels.graphics.tsaplots import plot_acf
import pandas_profiling

In [51]:
admin_df.Feedback_DateTime.dt.year

Feedback_ID
f72509b5-97c5-495b-8d71-1ad01dd2b955    2018
a9aad43c-609d-41f4-b36c-7174520923c9    2018
38426f17-f72a-49e0-b592-a933d2296678    2018
e0eb1cab-9357-4ae1-8629-96fc6eabd8b9    2018
0896dd84-e048-49a8-86b5-571e43245182    2018
                                        ... 
5bc1bf1b-6a25-409d-a987-094d61b61c9e    2020
b4668ee4-d38a-4186-8497-fd2ff1144af9    2020
d2adea03-783f-4451-b83d-d8cf3c3b1db9    2020
8f1dc7ac-d644-4763-b542-12d4511734e0    2020
898dc0fa-15ec-4c24-8021-094dc4ce0f35    2020
Name: Feedback_DateTime, Length: 576636, dtype: int64

In [52]:
admin_df = pd.read_csv('AdminSentimentData.csv',index_col='Feedback_ID') #From DataMart
admin_df['Feedback_DateTime'] = pd.to_datetime(admin_df['Feedback_DateTime'])
admin_df['month_year'] = admin_df.Feedback_DateTime.dt.to_period('M')
#admin_df = admin_df[admin_df.Feedback_DateTime.dt.year==]

In [53]:
# df = pd.read_excel('../TenantNPS/admin_verbatims_all_official_FY20filters_asof_20200305_ALL_THEMES.xlsx',index_col = 'Feedback_ID')
# df['Feedback_DateTime'] = pd.to_datetime(df['Feedback_DateTime'])
# df['month_year'] = df.Feedback_DateTime.dt.to_period('M')
# df.drop(columns='Unnamed: 0',inplace=True)
extra_col = pd.read_csv('../TenantNPS/all_admin_nps_feedbacks_with_totalusers.csv',index_col='Feedback_ID')
# admin_df = extra_col.join(df,how='left',rsuffix='O')
# admin_df.loc[admin_df['Feedback_Rating']==5,'Feedback_RatingValue']=100
# admin_df.loc[admin_df['Feedback_Rating']==4,'Feedback_RatingValue']=0
# admin_df.loc[admin_df['Feedback_Rating'].isin([1,2,3]),'Feedback_RatingValue']=-100

In [88]:
extra_col.head()

,Feedback_Rating,Feedback_DateTime,Tenant_TotalUsers
Feedback_ID,,,
1f2f78d9-04c8-4515-a1c7-1c1b58770817,1,2020-02-17 03:41:03.0000000,1
1fae113b-847f-4b10-98c4-7645cf54f719,5,2020-02-17 11:23:01.0000000,6
2754f3b6-851d-4cf9-9797-9e4997fc9c93,1,2020-02-17 04:20:06.0000000,144
32066075-9923-4166-bb8f-9135627cb4c2,4,2020-02-17 10:42:13.0000000,9
324b5116-65cf-4198-8e29-b2841c3a1283,2,2020-02-17 10:52:33.0000000,38


In [56]:
df_details = admin_df[admin_df.Verbatim_Details.notnull()]
Verbatim_Details = df_details.Verbatim_Details[admin_df.Verbatim_Details.notnull()]
Verbatim_Dates = df_details.month_year[admin_df.Verbatim_Details.notnull()]

In [57]:
verbatim_df = admin_df[admin_df.Feedback_Verbatim.notnull()]

In [58]:
verbatim_df.Verbatim_Themes

Feedback_ID
f116af54-22c4-4315-9f17-d4f0b0766fee               ["outlook"]
98d78aa6-b562-459b-a85a-e1598ad39f13            ["not mapped"]
0be16e62-f736-47a6-b535-94e3a3b40a1b            ["not mapped"]
952fb4a0-7201-4409-acda-049edcd44e2c      ["version platform"]
5c7ba6c5-644c-4f4f-9abd-cedac3729f93                 ["admin"]
                                                 ...          
fedd363c-cf80-41fc-9436-ad4bd323ff6a    ["subscription value"]
641a91e5-af0a-46ab-9d15-47bf6d87a36b            ["not mapped"]
08d9d345-c780-469d-98af-d860a24faa39            ["not mapped"]
02db4ce8-3206-469f-929f-642e4776953f             ["recommend"]
06da666e-55c2-490d-a2ee-31e22a6fadf9            ["not mapped"]
Name: Verbatim_Themes, Length: 43431, dtype: object

In [59]:
non_verbatim_df = admin_df[admin_df.Feedback_Verbatim.isnull()]
non_verbatim_df.shape

(533205, 8)

In [60]:
dictfile = pd.read_csv('tenant_nps_dict(commerce_update).csv')
dictfile['Labels'] = dictfile['Labels'].str.lower()
dictfile = dictfile.groupby(['Labels'])['Triggers'].apply(', '.join).reset_index()
dictionary_df = dictfile.groupby(['Labels'])['Triggers'].apply(', '.join).reset_index()
keys   = dictionary_df['Labels']
values = dictionary_df['Triggers'].str.split(',')
dictionary = {}
for k,v in zip(keys,values):
    dictionary[k.lower()]=v

import NewTopics
dictionary['compete'] = NewTopics.compete
dictionary['commerce'] = NewTopics.commerce

In [61]:
from NgramClassifier import NgramClassifier
import nps_uplift

In [62]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [72]:
Verbatim_Sample = verbatim_df.Verbatim_Details.values[2212]
j = json.loads(Verbatim_Sample)
j

['{"FeedbackSentencesId": 417484,"Sentence": "It is becoming MORE PROBLEMATIC to link my Outlook to Office 365 - THAT I HAVE SUBSCRIBED TO.", "AdminTheme":["outlook"], "SentimentClass": "Positive", "SentimentProbability": "0.998675"}',
 '{"FeedbackSentencesId": 417485,"Sentence": "IT IS FRUSTRATING.", "AdminTheme":["not mapped"], "SentimentClass": "Negative", "SentimentProbability": "0.047826"}']

In [76]:
sentences = [s.split(',')[:5] for s in j]
sentences[:3]

[['{"FeedbackSentencesId": 417484',
  '"Sentence": "It is becoming MORE PROBLEMATIC to link my Outlook to Office 365 - THAT I HAVE SUBSCRIBED TO."',
  ' "AdminTheme":["outlook"]',
  ' "SentimentClass": "Positive"',
  ' "SentimentProbability": "0.998675"}'],
 ['{"FeedbackSentencesId": 417485',
  '"Sentence": "IT IS FRUSTRATING."',
  ' "AdminTheme":["not mapped"]',
  ' "SentimentClass": "Negative"',
  ' "SentimentProbability": "0.047826"}']]

In [77]:
theme='outlook'

In [78]:
relevant_sentences = [l for l in sentences if theme in find_between(l[2],'["','"]')]
relevant_sentences

[['{"FeedbackSentencesId": 417484',
  '"Sentence": "It is becoming MORE PROBLEMATIC to link my Outlook to Office 365 - THAT I HAVE SUBSCRIBED TO."',
  ' "AdminTheme":["outlook"]',
  ' "SentimentClass": "Positive"',
  ' "SentimentProbability": "0.998675"}']]

In [79]:
Theme = find_between(relevant_sentences[0][0],'["','"]')
Class = find_between(relevant_sentences[0][1].split(':')[1],'"','"')
Prob = find_between(relevant_sentences[0][-1],":","}").strip().strip('"')

In [80]:
theme_sentiments = [find_between(relevant_sentences[x][2].split(':')[1],'"',
              '"') for x in range(len(relevant_sentences))]

'Negative' not in theme_sentiments

True

In [81]:
#in case Prob is a null value somewhere in data
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [82]:
for S in relevant_sentences:
    sentence_Theme = find_between(S[0],'["','"]')
    sentence_Class = find_between(S[1].split(':')[1],'"','"')
    sentence_Prob = find_between(S[-1],":","}").strip().strip('"')
    if Theme==theme:
        if isfloat(sentence_Prob):
            if sentence_Prob > 0.5:

SyntaxError: unexpected EOF while parsing (<ipython-input-82-65dcf34f9c2e>, line 7)

In [83]:
Verbatim_Details = admin_df.Verbatim_Details[admin_df.Verbatim_Details.notnull()]

In [84]:
details_dict = {}
for V, ix in zip(Verbatim_Details, Verbatim_Details.index):
    #Verbatim_Sample = Verbatim_Details.iloc[0]
    j = json.loads(V)
    themes = []

    for S in j:
        S = S.split(',')
        for AT in [item for item in S if item.find('AdminTheme')!=-1]:
            theme = find_between(AT,'["','"]')
            themes.append(theme)
    details_dict[ix] = ','.join(list(set(themes)))
            
details = pd.Series(details_dict)
details.fillna('',inplace=True)
details = details.str.replace('not mapped','')
details = details.str.replace('other','')
details = details.replace('no theme','')
details = details.str.strip(',')

i = 1
while sum(details.str.contains(',,'))>0:
    print('Time ',i)
    print('From ',sum(details.str.contains(',,')))
    details = details.str.replace(',,',',')
    print('To ',sum(details.str.contains(',,')))
    i+=1
    
details.replace('','no theme',inplace=True)
details = details.str.split(',').apply(set).apply(list)
details = details.astype(str).str.strip('[]')
details = details.str.replace(" '","'")
details = details.str.replace("'","")

Time  1
From  268
To  25
Time  2
From  25
To  0


In [85]:
admin_df['Verbatim_DetailsThemes'] = admin_df.index.map(details)

In [86]:
admin_df.Verbatim_DetailsThemes.fillna('no verbatim',inplace=True)

In [87]:
sentence_level_df = pd.DataFrame(columns=['FeedbackID','Sentence','MLTheme',
                                          'SentimentClass','SentimentProba']) #By Sentence, By Not Verbatim
for V, ix, Week in zip(Verbatim_Details, Verbatim_Details.index, Verbatim_Dates):
    j = json.loads(V)
    for S in j:
        l = S.split(',')
        #print(l,j,S)
        ID = int(l[0].split(':')[1].strip(" '"))
        Line = ','.join([x.rpartition('"Sentence": ')[2].strip('""') for x in l[1:-3]])
        #Line = sentence[1]
        Theme = find_between(l[-3],'["','"]')
        Class = find_between(l[-2].split(':')[1],'"','"')
        Prob = find_between(l[-1],":","}").strip().strip('"')
        sentence_level_df.loc[ID] = [ix,Line,Theme,Class,Prob]

In [89]:
sentence_level_df

,FeedbackID,Sentence,MLTheme,SentimentClass,SentimentProba
545692,f116af54-22c4-4315-9f17-d4f0b0766fee,#N#It was working with OWA but now the app is ...,outlook,Positive,0.927981
545691,f116af54-22c4-4315-9f17-d4f0b0766fee,It is very good on laptops but all my employee...,outlook,Positive,0.916740
345492,98d78aa6-b562-459b-a85a-e1598ad39f13,Very difficult,not mapped,Negative,0.242879
26404,0be16e62-f736-47a6-b535-94e3a3b40a1b,MS 365 is a powerful and reasonably easy to us...,not mapped,Negative,0.037441
337265,952fb4a0-7201-4409-acda-049edcd44e2c,Painful native apps on Mac,not mapped,Positive,0.576117
...,...,...,...,...,...
8413295,fedd363c-cf80-41fc-9436-ad4bd323ff6a,I very much hope that I can use your software ...,,,
8412975,641a91e5-af0a-46ab-9d15-47bf6d87a36b,ok,,,
8412765,08d9d345-c780-469d-98af-d860a24faa39,.,,,
8412747,02db4ce8-3206-469f-929f-642e4776953f,I already do that.,,,


make_uplit_df is the default Uplift Function

make_uplift_df2 is a revision that adjusts for passive/detractor feedback that actually contains "positive" sentiment for a given theme and is therefore not upliftable.

make_uplift_df3 is an previous revision with mobility for positive themes in detractors/passives

In [109]:
admin_df.shape

(576636, 9)

In [100]:
def make_uplift_df(input_df, theme_column='Verbatim_DetailsThemes', rating_column='Feedback_Rating'):
    """
    The input_df argument should contain a documents, ratings, and theme (document categories).
    The input_df should also contain non-verbatim responses from the same time period in order
    to calculate uplift.
    
    The theme_column is expected to have two special values: "no verbatim" and "no theme"; the 
    "no verbatim" value indicates that the feedback did not contain a verbatim, and the "no theme"
    value indicates that there is a verbatim but it did not get classified into a known theme
    category.  The theme_column should have all themes per verbatim in a single row, separated by
    commas.
    """
    
    # clean input_df
    input_df = input_df[(input_df[rating_column] >= 1) & (input_df[rating_column] <= 5)]
    
    # prepare verbatim only df
    input_df.loc[input_df['Feedback_Verbatim'].isnull(),theme_column] = input_df.loc[input_df['Feedback_Verbatim'].isnull(),theme_column].fillna('no verbatim') 
    input_df[theme_column].fillna('no theme',inplace=True)
    verbatim_df = input_df[input_df[theme_column] != 'no verbatim']
    
    # compute non-verbatim variables for formula
    non_verbatim_df = input_df[input_df[theme_column] == 'no verbatim']
    c = non_verbatim_df[non_verbatim_df[rating_column] == 5].shape[0]
    f = non_verbatim_df[non_verbatim_df[rating_column] == 4].shape[0]
    i = non_verbatim_df[non_verbatim_df[rating_column] <= 3].shape[0]
    
    theme_set = set()
    for csv_theme in verbatim_df[theme_column]:
        curr_themes = csv_theme.split(',')
        for theme in curr_themes:
            theme_set.add(theme)
            
    result_df = pd.DataFrame(columns=['Theme', 'Theme Count', 'Base NPS', 'NPS adjusted', 'NPS Max Uplift'])        
            
    result_idx = 0        
    for theme in theme_set:
        
        if theme == 'no theme':
            continue
        
        on_topic_df = verbatim_df[verbatim_df[theme_column].str.contains(theme)] #change to df[df[theme_column] == theme for Greg's stricter version]
        off_topic_df = verbatim_df[~verbatim_df[theme_column].str.contains(theme)] #change accordingly for stricter version
        assert on_topic_df.shape[0] + off_topic_df.shape[0] == verbatim_df.shape[0]
        
        # on-topic variables for formula
        a = on_topic_df[on_topic_df[rating_column] == 5].shape[0]
        d = on_topic_df[on_topic_df[rating_column] == 4].shape[0]
        g = on_topic_df[on_topic_df[rating_column] <= 3].shape[0]
        assert on_topic_df.shape[0] == a + d + g
        
        # off-topic variables for formula
        b = off_topic_df[off_topic_df[rating_column] == 5].shape[0]
        e = off_topic_df[off_topic_df[rating_column] == 4].shape[0]
        h = off_topic_df[off_topic_df[rating_column] <= 3].shape[0]
        assert off_topic_df.shape[0] == b + e + h
        
        movable_detractors = g + (g / (g+h) * i)
        movable_passives = d + (d / (d+e) * f)
        non_movable_detractors = h + (h / (g+h) * i)
        
        numer = a + b + c + movable_detractors + movable_passives - non_movable_detractors
        denom = np.sum([a,b,c,d,e,f,g,h,i])
        nps_adj = numer / denom * 100
        
        base_nps = ((a+b+c) - (g+h+i))/np.sum([a,b,c,d,e,f,g,h,i]) * 100
        
        result_df.loc[result_idx] = pd.Series({'Theme': theme, 'Theme Count':on_topic_df.shape[0], 'Base NPS': base_nps, 'NPS adjusted': nps_adj, 'NPS Max Uplift': nps_adj - base_nps})
        result_idx += 1
                
    return result_df
        
old_uplift_df = make_uplift_df(admin_df).sort_values(by='NPS Max Uplift',ascending=False) 

In [105]:
def make_uplift_df2(input_df, theme_column='Verbatim_DetailsThemes', rating_column='Feedback_Rating'):
    """
    The input_df argument should contain a documents, ratings, and theme (document categories).
    The input_df should also contain non-verbatim responses from the same time period in order
    to calculate uplift.
    
    The theme_column is expected to have two special values: "no verbatim" and "no theme"; the 
    "no verbatim" value indicates that the feedback did not contain a verbatim, and the "no theme"
    value indicates that there is a verbatim but it did not get classified into a known theme
    category.  The theme_column should have all themes per verbatim in a single row, separated by
    commas.
    """
    
    # clean input_df
    input_df = input_df[(input_df[rating_column] >= 1) & (input_df[rating_column] <= 5)]
    
    # prepare verbatim only df
    input_df.loc[input_df['Feedback_Verbatim'].isnull(),theme_column] = input_df.loc[input_df['Feedback_Verbatim'].isnull(),theme_column].fillna('no verbatim') 
    input_df[theme_column].fillna('no theme',inplace=True)
    verbatim_df = input_df[input_df[theme_column] != 'no verbatim']
    
    # compute non-verbatim variables for formula
    non_verbatim_df = input_df[input_df[theme_column] == 'no verbatim']
    c = non_verbatim_df[non_verbatim_df[rating_column] == 5].shape[0]
    f = non_verbatim_df[non_verbatim_df[rating_column] == 4].shape[0]
    i = non_verbatim_df[non_verbatim_df[rating_column] <= 3].shape[0]
    
    theme_set = set()
    for csv_theme in verbatim_df[theme_column]:
        curr_themes = csv_theme.split(',')
        for theme in curr_themes:
            theme_set.add(theme)
            
    result_df = pd.DataFrame(columns=['Theme', 'Theme Count', 'Base NPS', 'NPS adjusted', 'NPS Max Uplift'])        
            
    result_idx = 0        
    for theme in theme_set:
        #print(theme)
        if theme == 'no theme':
            continue
        
        on_topic_df = verbatim_df[verbatim_df[theme_column].str.contains(theme)] #change to df[df[theme_column] == theme for Greg's stricter version]
        off_topic_df = verbatim_df[~verbatim_df[theme_column].str.contains(theme)] #change accordingly for stricter version
        assert on_topic_df.shape[0] + off_topic_df.shape[0] == verbatim_df.shape[0]
        
        # on-topic variables for formula
        a = on_topic_df[on_topic_df[rating_column] == 5].shape[0] #|(on_topic_df['SentimentProba']>0.75)]?
        d_df = on_topic_df[on_topic_df[rating_column] == 4]#.shape[0]
        g_df = on_topic_df[on_topic_df[rating_column] <= 3]#.shape[0]
        #g_prime = 
                    
        #accounting for positives in Passives
        #current decision: remove
        for V in d_df.Verbatim_Details:
            j = json.loads(V)
            sentences = [s.split(',')[2:5] for s in j]
            relevant_sentences = [l for l in sentences if theme in find_between(l[0],'["','"]')]
            theme_sentiments = [find_between(relevant_sentences[x][1].split(':')[1],
                                             '"','"') for x in range(len(relevant_sentences))]
            before_removal = d_df.shape[0]
            if 'Positive' in theme_sentiments:
                #a = a.append(d[d.Verbatim_Details==V])
                d_df = d_df[d_df.Verbatim_Details!=V]
                assert before_removal > d_df.shape[0]
        
        #accounting for negatives in Promotors
        #current decision: do nothing 
        # for V in a.Verbatim_Details:
        #     j = json.loads(V)
        #     sentences = [s.split(',')[2:5] for s in j]
        #     relevant_sentences = [l for l in sentences if theme in find_between(l[0],'["','"]')]
        #     theme_sentiments = [find_between(relevant_sentences[x][1].split(':')[1],
        #                                      '"','"') for x in range(len(relevant_sentences))]
        #     if 'Negative' in theme_sentiments:
                #d = d.append(a[a.Verbatim_Details==V])
                #a = a[a.Verbatim_Details!=V]
        
        #accounting for positives in Detractors
        #current decision: remove
        #print(g_df.shape[0])
        for V in g_df.Verbatim_Details:
            j = json.loads(V)
            sentences = [s.split(',')[2:5] for s in j]
            relevant_sentences = [l for l in sentences if theme in find_between(l[0],'["','"]')]
            theme_sentiments = [find_between(relevant_sentences[x][1].split(':')[1],
                                             '"','"') for x in range(len(relevant_sentences))]
            before_removal = g_df.shape[0]
            if 'Positive' in theme_sentiments:
                #d = d.append(g[g.Verbatim_Details==V])
                g_df = g_df[g_df.Verbatim_Details!=V]
                assert before_removal > g_df.shape[0]
                
                
        #
        d = d_df.shape[0]
        g = g_df.shape[0]
        
        #assert on_topic_df.shape[0] == a + d + g
        
        # off-topic variables for formula
        b = off_topic_df[off_topic_df[rating_column] == 5].shape[0]
        e = off_topic_df[off_topic_df[rating_column] == 4].shape[0]
        h = off_topic_df[off_topic_df[rating_column] <= 3].shape[0]


        assert off_topic_df.shape[0] == b + e + h

        #print(a,b,c,d,e,f,g,h,i)
        
        movable_detractors = g + (g / (g+h) * i)
        movable_passives = d + (d / (d+e) * f)
        non_movable_detractors = h + (h / (g+h) * i)
        
        numer = a + b + c + movable_detractors + movable_passives - non_movable_detractors
        denom = np.sum([a,b,c,d,e,f,g,h,i])
        nps_adj = numer / denom * 100
        
        base_nps = ((a+b+c) - (g+h+i))/np.sum([a,b,c,d,e,f,g,h,i]) * 100
        
        result_df.loc[result_idx] = pd.Series({'Theme': theme, 'Theme Count':on_topic_df.shape[0], 'Base NPS': base_nps, 'NPS adjusted': nps_adj, 'NPS Max Uplift': nps_adj - base_nps})
        result_idx += 1
                
    return result_df
    

In [93]:
new_uplift_df = make_uplift_df2(admin_df).sort_values(by='NPS Max Uplift',ascending=False)

customer support
1463
704 14172 348673 174 5737 113658 1250 21121 70865
mobile
183
80 14796 348673 34 5935 113658 157 22401 70865
no theme
sharepoint
321
71 14805 348673 55 5909 113658 267 22263 70865
onedrive
415
141 14735 348673 93 5854 113658 358 22169 70865
yammer
5
0 14876 348673 5 5975 113658 5 22579 70865
marketing
1321
564 14312 348673 280 5588 113658 1123 21263 70865
skype
194
13 14863 348673 30 5943 113658 162 22390 70865
office client
374
64 14812 348673 70 5889 113658 321 22210 70865
outlook
1016
148 14728 348673 119 5817 113658 858 21568 70865
teams
40
18 14858 348673 5 5973 113658 26 22544 70865
exchange
233
120 14756 348673 40 5917 113658 180 22351 70865
commerce
958
102 14774 348673 79 5871 113658 847 21626 70865
admin
2575
572 14304 348673 319 5517 113658 2239 20009 70865


In [39]:
uplift_comparison = old_uplift_df.set_index('Theme').join(new_uplift_df.set_index('Theme'),
                                     lsuffix='(OLD)',rsuffix='(NEW)')

uplift_comparison['UpliftDelta'] = uplift_comparison['NPS Max Uplift(NEW)']-uplift_comparison['NPS Max Uplift(OLD)']
uplift_comparison.round(1)

,Theme Count(OLD),Base NPS(OLD),NPS adjusted(OLD),NPS Max Uplift(OLD),Theme Count(NEW),Base NPS(NEW),NPS adjusted(NEW),NPS Max Uplift(NEW),UpliftDelta
Theme,,,,,,,,,
admin,2635,46.7,52.1,5.4,2635,46.7,51.5,4.8,-0.7
marketing,1632,46.7,49.9,3.3,1632,46.7,49.5,2.8,-0.5
customer support,1759,46.7,49.7,3.0,1759,46.7,49.4,2.7,-0.4
outlook,922,46.7,48.7,2.0,922,46.7,48.5,1.8,-0.2
commerce,863,46.7,48.5,1.9,863,46.7,48.4,1.8,-0.1
onedrive,505,46.7,47.7,1.0,505,46.7,47.6,0.9,-0.1
office client,383,46.7,47.5,0.9,383,46.7,47.4,0.7,-0.1
sharepoint,340,46.7,47.4,0.7,340,46.7,47.3,0.6,-0.1
exchange,276,46.7,47.2,0.5,276,46.7,47.1,0.4,-0.1


In [ ]:
Step 1: Use the data from the CSR review - when you run old uplift

In [96]:
months_included = sorted(admin_df.month_year.dropna().unique())

In [102]:
data_dictionary = {}
for month in months_included:
    input_df = admin_df[admin_df['month_year']==month]
    print(input_df.shape)
    data = make_uplift_df(input_df,theme_column='Verbatim_DetailsThemes',rating_column='Feedback_Rating')
    data['month_year'] = str(month)
    data_dictionary[str(month)] = data
    
uplift_dfML = pd.concat([data_dictionary[str(x)] for x in months_included])

(18894, 9)
(17335, 9)
(21005, 9)
(33362, 9)
(30456, 9)
(25653, 9)
(36121, 9)
(29855, 9)
(29341, 9)
(27628, 9)
(30015, 9)
(21748, 9)
(23434, 9)
(25723, 9)
(44027, 9)
(59404, 9)
(40904, 9)
(33665, 9)
(28066, 9)


In [104]:
data_dictionary = {}
for month in months_included:
    input_df = admin_df[admin_df['month_year']==month]
    print(input_df.shape)
    data = make_uplift_df2(input_df,theme_column='Verbatim_DetailsThemes',rating_column='Feedback_Rating')
    data['month_year'] = str(month)
    data_dictionary[str(month)] = data

uplift_dfML2 = pd.concat([data_dictionary[str(x)] for x in months_included])

(18894, 9)
41
5
9
14
1
42
8
12
33
10
31
91
(17335, 9)
7
46
15
45
9
6
22
0
31
8
15
70
(21005, 9)
59
15
14
11
49
14
5
45
0
9
27
88
(33362, 9)
19
96
15
71
20
16
57
3
41
16
16
149
(30456, 9)
68
11
24
58
21
9
70
1
44
19
15
122
(25653, 9)
61
5
5
12
0
64
13
9
51
0
9
39
105
(36121, 9)
100
6
20
23
0
68
21
19
68
2
16
55
126
(29855, 9)
71
5
15
19
56
18
9
65
2
22
44
92
(29341, 9)
72
7
16
21
56
24
2
64
5
11
38
132
(27628, 9)
72
7
11
24
0
56
20
9
42
9
39
113
(30015, 9)
78
10
21
67
14
16
43
5
40
8
11
125
(21748, 9)
46
5
10
20
54
10
14
34
4
8
32
83
(23434, 9)
48
5
15
21
48
5
20
37
1
8
42
121
(25723, 9)
81
10
1
33
79
2
12
59
23
47
19
26
135
(44027, 9)
140
18
1
43
32
145
42
23
89
7
25
92
277
(59404, 9)
96
11
31
87
16
11
68
4
86
4
31
236
(40904, 9)
100
10
1
22
14
85
3
13
66
23
10
102
194
(33665, 9)
133
19
1
38
131
32
10
76
3
88
15
35
229
(28066, 9)
8
55
8
60
16
1
27
1
40
7
11
87


In [106]:
ml_to_trend = ['admin','commerce','customer support','outlook']

In [107]:
MLuplift_monthly = uplift_dfML[uplift_dfML.Theme.isin(ml_to_trend)].pivot(index='month_year',
                                                                          columns='Theme',values='NPS Max Uplift')

In [108]:
MLuplift_monthly

Theme,admin,commerce,customer support,outlook
month_year,,,,
2018-07,6.224723,2.221437,3.058731,2.320864
2018-08,5.491332,1.809811,3.571862,1.973755
2018-09,6.493470,1.459341,3.398910,3.291240
2018-10,6.201653,1.560451,3.862485,2.264898
2018-11,5.646474,1.886924,3.175248,3.140108
2018-12,6.386217,1.888967,2.831038,2.938986
2019-01,5.975769,2.253227,4.162762,2.917780
2019-02,4.556603,1.820179,3.285681,3.189417
2019-03,5.665661,1.719772,3.215163,3.132487


In [66]:
admin_df['Verbatim_Themes2'] = admin_df['Verbatim_Themes'].str.strip('[]*"')
admin_df['Verbatim_Themes2'] = admin_df['Verbatim_Themes2'].str.replace('","',',')
admin_df['Verbatim_Themes2'].replace('not mapped','no theme',inplace=True)

In [67]:
old_uplift_df = make_uplift_df(admin_df,theme_column = 'Verbatim_Themes2').sort_values(by='NPS Max Uplift',ascending=False) 
new_uplift_df = make_uplift_df2(admin_df, theme_column = 'Verbatim_Themes2').sort_values(by='NPS Max Uplift',ascending=False)

uplift_comparison = old_uplift_df.set_index('Theme').join(new_uplift_df.set_index('Theme'),
                                     lsuffix='(OLD)',rsuffix='(NEW)')

uplift_comparison['UpliftDelta'] = uplift_comparison['NPS Max Uplift(NEW)']-uplift_comparison['NPS Max Uplift(OLD)']
uplift_comparison.round(1)

authentication
645
28 10945 242451 48 4228 78302 645 15234 49984
skype for business
673
128 10845 242451 131 4145 78302 673 15206 49984
internet explorer
23
4 10969 242451 5 4271 78302 23 15856 49984
easeofuse-popup
1199
52 10921 242451 50 4226 78302 1199 14680 49984
excel
227
48 10925 242451 50 4226 78302 227 15652 49984
admin
2411
807 10166 242451 501 3739 78302 2342 13468 49984
groups
112
38 10935 242451 32 4244 78302 112 15767 49984
easeofuse-navigation
350
34 10939 242451 54 4222 78302 350 15529 49984
recommend
1215
440 10533 242451 236 4040 78302 1215 14664 49984
version platform
234
41 10932 242451 44 4232 78302 234 15645 49984
teams
333
100 10873 242451 88 4187 78302 325 15546 49984
easeofuse-version
772
247 10726 242451 192 4084 78302 772 15107 49984
powerpoint
183
49 10924 242451 40 4236 78302 183 15696 49984
compete
505
87 10886 242451 92 4184 78302 505 15374 49984
word
284
49 10924 242451 64 4212 78302 284 15595 49984
ms online
1
0 10973 242451 0 4276 78302 1 15878 49984
ex

,Theme Count(OLD),Base NPS(OLD),NPS adjusted(OLD),NPS Max Uplift(OLD),Theme Count(NEW),Base NPS(NEW),NPS adjusted(NEW),NPS Max Uplift(NEW),UpliftDelta
Theme,,,,,,,,,
ease_of_use,6722,46.7,60.7,14.0,6722,46.7,60.7,14.0,0.0
admin,3755,46.7,54.2,7.6,3755,46.7,54.0,7.3,-0.3
outlook,3115,46.7,53.3,6.6,3115,46.7,53.1,6.4,-0.2
subscription value,2647,46.7,51.8,5.1,2647,46.7,51.8,5.1,0.0
easeofuse-generalcomplexity,2469,46.7,51.6,4.9,2469,46.7,51.6,4.9,0.0
support,2437,46.7,51.1,4.4,2437,46.7,50.8,4.1,-0.3
account,1841,46.7,50.4,3.7,1841,46.7,50.4,3.7,0.0
recommend,1891,46.7,50.3,3.6,1891,46.7,50.3,3.6,0.0
easeofuse-howto,1280,46.7,49.6,3.0,1280,46.7,49.6,3.0,0.0


In [188]:
def make_uplift_df3(input_df, theme_column='Verbatim_DetailsThemes', rating_column='Feedback_Rating'):
    """
    The input_df argument should contain a documents, ratings, and theme (document categories).
    The input_df should also contain non-verbatim responses from the same time period in order
    to calculate uplift.
    
    The theme_column is expected to have two special values: "no verbatim" and "no theme"; the 
    "no verbatim" value indicates that the feedback did not contain a verbatim, and the "no theme"
    value indicates that there is a verbatim but it did not get classified into a known theme
    category.  The theme_column should have all themes per verbatim in a single row, separated by
    commas.
    """
    
    # clean input_df
    input_df = input_df[(input_df[rating_column] >= 1) & (input_df[rating_column] <= 5)]
    
    # prepare verbatim only df
    verbatim_df = input_df[input_df[theme_column] != 'no verbatim']
    verbatim_df[theme_column].fillna(value='no theme', inplace=True)
    
    # compute non-verbatim variables for formula
    non_verbatim_df = input_df[input_df[theme_column] == 'no verbatim']
    c = non_verbatim_df[non_verbatim_df[rating_column] == 5].shape[0]
    f = non_verbatim_df[non_verbatim_df[rating_column] == 4].shape[0]
    i = non_verbatim_df[non_verbatim_df[rating_column] <= 3].shape[0]
    
    theme_set = set()
    for csv_theme in verbatim_df[theme_column]:
        curr_themes = csv_theme.split(',')
        for theme in curr_themes:
            theme_set.add(theme)
            
    result_df = pd.DataFrame(columns=['Theme', 'Theme Count', 'Base NPS', 'NPS adjusted', 'NPS Max Uplift'])        
            
    result_idx = 0        
    for theme in theme_set:
        if theme == 'no theme':
            continue
        
        on_topic_df = verbatim_df[verbatim_df[theme_column].str.contains(theme)] #change to df[df[theme_column] == theme for Greg's stricter version]
        off_topic_df = verbatim_df[~verbatim_df[theme_column].str.contains(theme)] #change accordingly for stricter version
        assert on_topic_df.shape[0] + off_topic_df.shape[0] == verbatim_df.shape[0]
        
        # on-topic variables for formula
        a = on_topic_df[on_topic_df[rating_column] == 5]#.shape[0] #|(on_topic_df['SentimentProba']>0.75)]?
        d = on_topic_df[on_topic_df[rating_column] == 4]#.shape[0]
        g = on_topic_df[on_topic_df[rating_column] <= 3]#.shape[0]
        
                    
        #accounting for positives in Passives
        #current decision: move up 1 class
        for V in d.Verbatim_Details:
            j = json.loads(V)
            sentences = [s.split(',')[2:5] for s in j]
            relevant_sentences = [l for l in sentences if theme in find_between(l[0],'["','"]')]
            theme_sentiments = [find_between(relevant_sentences[x][1].split(':')[1],
                                             '"','"') for x in range(len(relevant_sentences))]
            if 'Negative' not in theme_sentiments:
                a = a.append(d[d.Verbatim_Details==V])
                d = d[d.Verbatim_Details!=V]
        
        #accounting for negatives in Promotors
        #current decision: move down 1 class
        for V in a.Verbatim_Details:
            j = json.loads(V)
            sentences = [s.split(',')[2:5] for s in j]
            relevant_sentences = [l for l in sentences if theme in find_between(l[0],'["','"]')]
            theme_sentiments = [find_between(relevant_sentences[x][1].split(':')[1],
                                             '"','"') for x in range(len(relevant_sentences))]
            if 'Negative' in theme_sentiments:
                d = d.append(a[a.Verbatim_Details==V])
                a = a[a.Verbatim_Details!=V]
        
        #accounting for positives in Detractors
        #current decision: move up 1 class
        for V in g.Verbatim_Details:
            j = json.loads(V)
            sentences = [s.split(',')[2:5] for s in j]
            relevant_sentences = [l for l in sentences if theme in find_between(l[0],'["','"]')]
            theme_sentiments = [find_between(relevant_sentences[x][1].split(':')[1],
                                             '"','"') for x in range(len(relevant_sentences))]
            if 'Negative' not in theme_sentiments:
                d = d.append(g[g.Verbatim_Details==V])
                g = g[g.Verbatim_Details!=V]
                
                
        a = a.shape[0] #|(on_topic_df['SentimentProba']>0.75)]?
        d = d.shape[0]
        g = g.shape[0]
        
        assert on_topic_df.shape[0] == a + d + g
        
        # off-topic variables for formula
        b = off_topic_df[off_topic_df[rating_column] == 5].shape[0]
        e = off_topic_df[off_topic_df[rating_column] == 4].shape[0]
        h = off_topic_df[off_topic_df[rating_column] <= 3].shape[0]


        assert off_topic_df.shape[0] == b + e + h
        
        movable_detractors = g + (g / (g+h) * i)
        movable_passives = d + (d / (d+e) * f)
        non_movable_detractors = h + (h / (g+h) * i)
        
        numer = a + b + c + movable_detractors + movable_passives - non_movable_detractors
        denom = np.sum([a,b,c,d,e,f,g,h,i])
        nps_adj = numer / denom * 100
        
        base_nps = ((a+b+c) - (g+h+i))/np.sum([a,b,c,d,e,f,g,h,i]) * 100
        
        result_df.loc[result_idx] = pd.Series({'Theme': theme, 'Theme Count':on_topic_df.shape[0], 'Base NPS': base_nps, 'NPS adjusted': nps_adj, 'NPS Max Uplift': nps_adj - base_nps})
        result_idx += 1
                
    return result_df

alt_uplift_df = make_uplift_df3(admin_df).sort_values(by='NPS Max Uplift',ascending=False)


In [191]:
uplift_comparison2 = new_uplift_df.set_index('Theme').join(alt_uplift_df.set_index('Theme'),
                                     lsuffix='(New)',rsuffix='(Alt)')

uplift_comparison2['UpliftDelta'] = uplift_comparison2['NPS Max Uplift(Alt)']-uplift_comparison2['NPS Max Uplift(New)']

In [195]:
uplift_comparison2.round(1)

,Theme Count(New),Base NPS(New),NPS adjusted(New),NPS Max Uplift(New),Theme Count(Alt),Base NPS(Alt),NPS adjusted(Alt),NPS Max Uplift(Alt),UpliftDelta
Theme,,,,,,,,,
admin,2635,46.7,52.2,5.5,2635,46.9,53.1,6.2,0.7
marketing,1632,46.7,50.0,3.3,1632,46.8,50.3,3.5,0.2
customer support,1759,46.7,49.8,3.1,1759,46.8,50.4,3.6,0.5
outlook,922,46.7,48.8,2.1,922,46.8,49.2,2.5,0.4
commerce,863,46.7,48.6,1.9,863,46.7,49.1,2.3,0.4
onedrive,505,46.7,47.7,1.1,505,46.7,47.9,1.2,0.1
office client,383,46.7,47.6,0.9,383,46.7,47.8,1.1,0.2
sharepoint,340,46.7,47.5,0.8,340,46.7,47.7,0.9,0.1
exchange,276,46.7,47.3,0.6,276,46.7,47.4,0.7,0.1


In [ ]:
uplift_comparison2 = new_uplift_df.set_index('Theme').join(alt_uplift_df.set_index('Theme'),
                                     lsuffix='(New)',rsuffix='(Alt)')

In [204]:
uplift_comparison3 = old_uplift_df.set_index('Theme').join(alt_uplift_df.set_index('Theme'),
                                     lsuffix='(Old)',rsuffix='(Alt)')

uplift_comparison3['UpliftDelta'] = uplift_comparison3['NPS Max Uplift(Alt)']-uplift_comparison3['NPS Max Uplift(Old)']
uplift_comparison3.round(1)

,Theme Count(Old),Base NPS(Old),NPS adjusted(Old),NPS Max Uplift(Old),Theme Count(Alt),Base NPS(Alt),NPS adjusted(Alt),NPS Max Uplift(Alt),UpliftDelta
Theme,,,,,,,,,
admin,2635,46.7,52.1,5.4,2635,46.9,53.1,6.2,0.8
marketing,1632,46.7,49.9,3.3,1632,46.8,50.3,3.5,0.3
customer support,1759,46.7,49.7,3.0,1759,46.8,50.4,3.6,0.6
outlook,922,46.7,48.7,2.0,922,46.8,49.2,2.5,0.5
commerce,863,46.7,48.5,1.9,863,46.7,49.1,2.3,0.5
onedrive,505,46.7,47.7,1.0,505,46.7,47.9,1.2,0.1
office client,383,46.7,47.5,0.9,383,46.7,47.8,1.1,0.2
sharepoint,340,46.7,47.4,0.7,340,46.7,47.7,0.9,0.2
exchange,276,46.7,47.2,0.5,276,46.7,47.4,0.7,0.1


In [205]:
uplift_comparison3.to_csv('SentimentUpliftEffect.csv')

In [206]:
'102'[::-1]

'201'

In [65]:
coins

7